## Preparation

In [2]:
import os
import glob
import datetime
import geopandas as gpd
import pandas as pd

In [3]:
# read inputs from spreadsheet
working_dr = os.getcwd()
data_dr = os.path.join(working_dr, 'data')
inputs_dr = os.path.join(working_dr, 'inputs.csv')
local_images_dr = os.path.join(working_dr, 'local data')
inputs_df = pd.read_csv(inputs_dr)

In [4]:
# choose a season

# by index
index = 0
row = inputs_df.iloc[index]

# # by name
# name = '2020_2021_wheat'
# row = inputs_df.loc[inputs_df['season'] == name].iloc[0]

season = row['season']
project_area_dr = row['project_area']
field_area_dr = row['field_area']
PET_area_dr = row['PET_area']
start_date = row['start_date']
end_date = row['end_date']

In [5]:
# convert shapefile to featurecollection
project_gdf = gpd.read_file(project_area_dr) #read shapefile
field_gdf = gpd.read_file(field_area_dr) #read shapefile
PET_gdf = gpd.read_file(PET_area_dr) #read shapefile
bbox = project_gdf.total_bounds.tolist()

## Get (ETa, NDVI, RGB) geeSEBAL for each landsat image

In [6]:
import ee
# Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

aoi = ee.Geometry.Rectangle(bbox)
geojson = field_gdf.__geo_interface__
aoi_field = ee.FeatureCollection(geojson['features'])

In [13]:
from geesebal import Image

# landsat images
ls8_imgs = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
  .filterBounds(aoi)\
  .filter(ee.Filter.date(start_date, end_date)).distinct(['DATE_ACQUIRED'])
ls9_imgs = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')\
  .filterBounds(aoi)\
  .filter(ee.Filter.date(start_date, end_date)).distinct(['DATE_ACQUIRED'])
result_imgs = []
ls8_list_size = ls8_imgs.size().getInfo()
ls8_list = ls8_imgs.toList(ls8_list_size)
print('ls8 images')
for i in range(ls8_list_size):
    img = ee.Image(ls8_list.get(i))
    landsat_img = Image(img.get('system:id').getInfo())
    result_imgs.append(landsat_img)
    print(f"{i+1} / {ls8_list_size} done")
print('ls8 done')

ls9_list_size = ls9_imgs.size().getInfo()
ls9_list = ls9_imgs.toList(ls9_list_size)
print('ls9 images')
for i in range(ls9_list_size):
    img = ee.Image(ls9_list.get(i))
    landsat_img = Image(img.get('system:id').getInfo(),
                NDVI_cold=5,
                Ts_cold=20,
                NDVI_hot=1,
                Ts_hot=10)
    result_imgs.append(landsat_img)
    print(f"{i+1} / {ls9_list_size} done")
print('ls9 done')

ls8 images
1 / 2 done
2 / 2 done
ls8 done
ls9 images
1 / 3 done
2 / 3 done
3 / 3 done
ls9 done


In [14]:
# for local images
import glob
from sebal import Image_local

local_images = [os.path.split(i)[0] for i in glob.glob(os.path.join(local_images_dr, 'landsat images')+'/*'+'/*/')]
for local_image in local_images:
  landsat_img = Image_local(local_image)

: 

### export tifs (RGB, NDIV, ETa)

In [14]:
shp_name = os.path.basename(project_area_dr).split('.')[0]
project_dr = os.path.join(working_dr, 'data', shp_name)
season_dr = os.path.join(working_dr, 'data', shp_name, f'season_{season}')
et_folder = os.path.join(season_dr, 'ET')
ndvi_folder = os.path.join(season_dr, 'ndvi')
rgb_folder = os.path.join(season_dr, 'rgb')

In [10]:
# for gee images
from funcs.export import export_to_drive, export_gee_tifs_localy

In [ ]:
# # export to drive (gee)
# shp_name = os.path.basename(project_shp_location).split('.')[0]
# for img in result_imgs:
#     # get img date
#     date = img.date_string
#     # export tif (to drive)
#     export_to_drive(img, date, os.path.join(shp_name, f'season_{season}'), aoi)

In [16]:
# export localy (gee)
for img in result_imgs:
  try:
    date = img.date_string
    export_gee_tifs_localy(img.image, 'RGB', 'rgb', season_dr, date, aoi)
    export_gee_tifs_localy(img.image, 'ET_24h', 'eta', season_dr, date, aoi)
    export_gee_tifs_localy(img.image, 'NDVI', 'ndvi', season_dr, date, aoi)
  except Exception as error:
    print(img.landsat_version)
    print(img.date_string)
    print(f"Exception = {error}")

In [ ]:
from funcs.export import export_local_tifs
# export local tifs
data_drs = glob.glob(os.path.join(local_images_dr, 'results')+"/*")
export_local_tifs(data_drs, season_dr, project_gdf)

## Get AETI and RET from WaPOR data

In [2]:
from funcs.export import download_WaPOR

In [28]:
period = [start_date, end_date]
overview = "NONE"
variables = ["L2-AETI-D", "L2-NPP-D"]
region = project_area_dr
download_WaPOR(region, variables, period, season_dr, overview)
variables = ["L1-RET-E"]
region = PET_area_dr
download_WaPOR(region, variables, period, season_dr, overview)

INFO: Found 3 files for L2-AETI-D.
INFO: Converting units from [mm/day] to [mm/dekad].
INFO: Splitting single GeoTIFF into multiple unscaled files.
INFO: Found 45 files for L1-RET-E.
INFO: Units are already as requested, no conversion needed.
INFO: Splitting single GeoTIFF into multiple unscaled files.


## Daily ET dataframe

In [13]:
from funcs.ETp import ETp_list

In [ ]:
# Get ETp for each date
# season days
s_d = datetime.datetime.strptime(start_date,'%Y-%m-%d')
e_d = datetime.datetime.strptime(end_date,'%Y-%m-%d')
days = [(s_d + datetime.timedelta(days=x)).strftime('%Y-%m-%d') for x in range((e_d-s_d).days + 2)]
daily_ETp_list = ETp_list(days, aoi)
days = days[:-1]
daily_ET_df = pd.DataFrame(
  {'date': pd.to_datetime(days) ,'ETp': daily_ETp_list, 'kc': np.nan, 'ETa': np.nan}
  )
os.makedirs(os.path.join(et_folder, "datasheets"), exist_ok= True)
daily_ET_df.to_csv(os.path.join(et_folder, "datasheets", "daily_ET.csv"))